In [24]:
import tensorflow as tf
from tensorboard.plugins.hparams import api as hp
from tensorflow.keras.optimizers import Adam
import ast
#trouver un moyen de rapidment eleminer certaines valeurs
# Define the hyperparameters
#utilise pas tout mais definis un range et lesquels sont possible

#cles du dict hparams
HP_ARCHITECTURE = hp.HParam('architecture',
                              hp.Discrete(['[5, 39, 40, 31, 42, 1]',
                                           '[5, 61, 51, 28, 39, 26, 21,20, 14, 1]']))
HP_LEARNING_RATE = hp.HParam('learning_rate', 
                             hp.Discrete([i/1000 for i in range(1,10,1)]))
HP_DROPOUT_RATE = hp.HParam('drouptout_rate',
                            hp.Discrete([i/10 for i in range(1,10,1)]))
HP_LRDECAY_RATE = hp.HParam('lrDecay_rate', 
                            hp.Discrete([i/100 for i in range(60,100,10)]))
HP_RYTHM_RATE = hp.HParam('rythm', 
                          hp.Discrete([i for i in range(10,30,10)]))
HP_ALPHA_ACTI = hp.HParam('alpha_acti', 
                          hp.Discrete([i/50 for i in range(0,10,1)]))
HP_BATCH_SIZE = hp.HParam('batch_size', 
                          hp.Discrete([16,32,64]))
HP_BATCHNORM = hp.HParam('batch_normalisation', 
                         hp.Discrete([True, False]))


HP_SEED = hp.HParam('seed', 
                    hp.Discrete([0,1,2]))

HP_OPTIMIZER = hp.HParam('optimizer', 
                         hp.Discrete(['adam','sgd']))

def nrmse(y_true, y_pred):
    return K.sqrt(K.mean(K.square(y_pred - y_true)))/K.mean(y_true)
    
myMetrics = [hp.Metric('msle', display_name='msle'), 
             hp.Metric('mape', display_name='mape')]#,
             #hp.Metric(nrmse, display_name='nrmse')]



# Configure the hyperparameters and metrics for TensorBoard
with tf.summary.create_file_writer('logs/hparam_tuning').as_default():
    hp.hparams_config(
        hparams=[
            HP_ARCHITECTURE, 
            HP_LEARNING_RATE,
            HP_DROPOUT_RATE,
            HP_LRDECAY_RATE,
            HP_RYTHM_RATE,
            HP_ALPHA_ACTI,
            HP_BATCH_SIZE,
            HP_BATCHNORM,
            HP_SEED,
            HP_OPTIMIZER
            ],
        metrics=myMetrics,
    )
    

ValueError: Unknown dtype: <class 'type'>

In [ ]:

def train_test_model(hparams):
  model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(hparams[HP_NUM_UNITS], activation=tf.nn.relu),
    tf.keras.layers.Dropout(hparams[HP_DROPOUT]),
    tf.keras.layers.Dense(10, activation=tf.nn.softmax),
  ])
  model.compile(
      optimizer=hparams[HP_OPTIMIZER],
      loss='sparse_categorical_crossentropy',
      metrics=['accuracy'],
  )

  model.fit(x_train, y_train, epochs=1) # Run with 1 epoch to speed things up for demo purposes
  _, accuracy = model.evaluate(x_test, y_test)
  return accuracy



def run(run_dir, hparams):
  with tf.summary.create_file_writer(run_dir).as_default():
    hp.hparams(hparams)  # record the values used in this trial
    accuracy = train_test_model(hparams)
    tf.summary.scalar(METRIC_ACCURACY, accuracy, step=1)
      
model.fit(
    ...,
    callbacks=[
        tf.keras.callbacks.TensorBoard(logdir),  # log metrics
        hp.KerasCallback(logdir, hparams),  # log hparams
    ],
)

In [ ]:
session_num = 0

for num_units in HP_NUM_UNITS.domain.values:
  for dropout_rate in (HP_DROPOUT.domain.min_value, HP_DROPOUT.domain.max_value):
    for optimizer in HP_OPTIMIZER.domain.values:
      hparams = {
          HP_NUM_UNITS: num_units,
          HP_DROPOUT: dropout_rate,x
          HP_OPTIMIZER: optimizer,
      }
      run_name = "run-%d" % session_num
      print('--- Starting trial: %s' % run_name)
      print({h.name: hparams[h] for h in hparams})
      run('logs/hparam_tuning/' + run_name, hparams)
      session_num += 1



In [ ]:
#ou alors

"""

# Define your hyperparameters
learning_rate = 0.001
units = 64

# Create a summary writer and start a new session
writer = tf.summary.create_file_writer('logs/hparams')

# Define the hyperparameter configuration
hyperparams = {
    'learning_rate': learning_rate,
    'units': units
}

# Log the hyperparameters
with writer.as_default():
    hp.hparams(hyperparams)  # Log the hyperparameters

"""
# Start the TensorBoard server to visualize the logs
# Open a terminal or command prompt and navigate to the directory containing your code
# Run the fol